# Using ctypes, a foreign function library for Python.
https://docs.python.org/3.5/library/ctypes.html

## 1. execute a function from an system library

In [1]:
! man 2 time

TIME(2)                    Linux Programmer's Manual                   TIME(2)

NAME
       time - get time in seconds

SYNOPSIS
       #include <time.h>

       time_t time(time_t *tloc);

DESCRIPTION
       time()  returns  the  time  as  the  number of seconds since the Epoch,
       1970-01-01 00:00:00 +0000 (UTC).

       If tloc is non-NULL, the return value is  also  stored  in  the  memory
       pointed to by tloc.

RETURN VALUE
       On  success,  the value of time in seconds since the Epoch is returned.
       On error, ((time_t) -1) is returned, and errno is set appropriately.

ERRORS
       EFAULT tloc points outside  your  accessible  address  space  (but  see
              BUGS).

              On  systems  where the C library time() wrapper function invokes
              an implementation provided by the vdso(7) (so that there  is  no
              trap  into the kernel), an invalid address may instead trigger a
              SIGSEGV signal.

CONFORMING TO
       SVr4,

In [5]:
from ctypes import CDLL

libc = CDLL("libc.so.6") 
print(libc.time(None))

1496779469


Because of the whole LD_LIBRARY_PATH setting requirement, ctypes is really intended for accessing system libraries. 

But you can still use it for other stuff.

## 2. execute a function from your own (C) library

### 2.1 by passing parameter by value

#### Header file sum.h

In [ ]:
#ifndef __REDUCTION_H__
#define  __REDUCTION_H__
    float sum(float a, float b);
#endif

#### C file sum.c

In [ ]:
float sum(float a, float b){
    float result = a + b;
    return result;
}

#### commands to build the shared library

In [7]:
! gcc -fPIC -Wall -c sum.c -I. -o sum.o
! gcc -shared  -o libsum.so sum.o
! export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:.

In [8]:
from ctypes import CDLL
from ctypes import c_float

lib = CDLL("libsum.so")
lib.sum.restype = c_float
lib.sum.argtypes = [c_float, c_float]

a = 3.43
b = 2.65
result =  lib.sum(a, b)
print('{} +  {} = {:.2f}'.format(a, b, result))

3.43 +  2.65 = 6.08


### 2.2 by passing parameter by address (~by reference)

#### header array_sum.h

In [ ]:
#ifndef __REDUCTION_H__
#define  __REDUCTION_H__

float array_sum(float* tab, unsigned int size);

#endif

#### C file array_sum.c

In [ ]:
/// Sum of array elements.
/**	@param tab  : float pointer ,    array to sum
 * 	@param size : unsigned int  ,    array size
 * 	@return     : float         ,    sum of array elements
*/
float array_sum(float* tab, unsigned int size){

    float result = 0.;
        for ( unsigned int i =0; i < size; ++i){
            result += tab[i];
        }
        return result;
}


In [9]:
! gcc -fPIC -Wall -c array_sum.c -I. -o array_sum.o
! gcc -shared  -o libarraysum.so array_sum.o
! ls

array_sum.c  array_sum.o     libsum.so	   sum.c  sum.o
array_sum.h  libarraysum.so  simple.ipynb  sum.h


In [10]:
from ctypes import CDLL
from ctypes import c_float
from ctypes import c_uint
from ctypes import POINTER

lib = CDLL("libarraysum.so")
lib.array_sum.restype = c_float
lib.array_sum.argtypes = [POINTER(c_float),c_uint]

l = [1,2,3,4,5]
arr = (c_float * len(l))(*l)
result = lib.array_sum(arr, len(l))
print('sum {} = {:.2f}'.format(l, result))

sum [1, 2, 3, 4, 5] = 15.00


Note : a star before a list will unpack it
`l = [1,2,3,4,5]`
`arr = (c_float * len(l))(*l)`
is equivalent to
`arr = (c_float * len(l))(1,2,3,4,5)`